In [19]:
import os
import pandas as pd
from pathlib import Path
import numpy as np
import datetime as dt 

#### Data taxis verdes

In [13]:
# cargar datos
def load_parquet_files(folder_path, Año):
    parquet_files = Path(folder_path).glob('*.parquet')#Obtenemos una lista de rutas a archivos .parquet en la carpeta especificada
    parquet_files_Año = [file for file in parquet_files if f'green_tripdata_{Año}' in file.name]#filtramos el nombre del archivo con el que vamos trabajar

    data_frames_Año = []#iniciamos un lista para almacenar los DataFrames cargados desde los archivos .parquet
    for file in parquet_files_Año:# Iteramos a través de los archivos .parquet correspondientes al año y cargar cada uno en un DataFrame
        df = pd.read_parquet(file)# Cargamos el archivo parquet
        data_frames_Año.append(df)# Agregamos el DataFrame cargado a la lista

    return data_frames_Año # Devolvemos la lista de DataFrames correspondientes al año


In [20]:
#tranformar datos
def transform_data(data_frames, Año):
    combined_df = pd.concat(data_frames, ignore_index=True)
    combined_df['lpep_pickup_datetime'] = pd.to_datetime(combined_df['lpep_pickup_datetime'])
    combined_df['pickup_date'] = combined_df['lpep_pickup_datetime'].dt.date

    df_taxi = combined_df.copy()
    df_taxi['pickup_date'] = pd.to_datetime(df_taxi['pickup_date'])

    column_order = ['VendorID', 'pickup_date'] + [col for col in df_taxi.columns if col not in ['VendorID', 'pickup_date']]
    df_taxi = df_taxi[column_order]

    df_taxi = df_taxi[df_taxi['pickup_date'].dt.year == Año]

    columns_to_remove = ['lpep_pickup_datetime','lpep_dropoff_datetime','RatecodeID','PULocationID','DOLocationID','passenger_count','trip_distance','fare_amount','payment_type','trip_type','extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'ehail_fee', 'improvement_surcharge', 'store_and_fwd_flag', 'congestion_surcharge']
    df_taxi = df_taxi.drop(columns=columns_to_remove)

    # Agregar una columna que indica el día de la semana (0: Lunes, 6: Domingo)
    df_taxi['weekday'] = df_taxi['pickup_date'].dt.weekday

    # Mapear los números de día de la semana a nombres de días
    day_names = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    df_taxi['weekday'] = df_taxi['weekday'].apply(lambda x: day_names[x])

    # Agrupar por fecha y calcular el total_amount por día
    df_taxi_grouped = df_taxi.groupby(['pickup_date', 'weekday'])['total_amount'].sum().reset_index()

    # Agregar una columna que indica la cantidad de veces que se repite una fecha
    df_taxi_grouped['row_count'] = df_taxi['pickup_date'].value_counts()[df_taxi_grouped['pickup_date']].values

    return df_taxi_grouped

Realizamos los llamados a las funciones 

In [21]:

folder_path = 'Datasets\Dataset_extraidos\Taxis.parquet_2022_2023'
Año = 2022
data_frames_Año = load_parquet_files(folder_path, Año)
df_taxi_grouped = transform_data(data_frames_Año, Año)
df_taxi_grouped.head(10)
#Guardamos el DataFrame con el nombre correspondiente al año elegido
df_taxi_grouped.to_csv(f'taxiV_{Año}.csv') 


ValueError: No objects to concatenate

#### Data Taxis amarillos

Se realiza el mismo procedimiento que en la funcion anterior, con la diferencia que al ser archivos mas pesados se deben crear DataFrame por día

Función para la carga de datos

In [24]:
def load_parquet_filesY(folder_path, AñoY):
    parquet_files = Path(folder_path_Y).glob('*.parquet')
    parquet_files_Año = [file for file in parquet_files if f'yellow_tripdata_{AñoY}-0{mes}' in file.name]

    data_frames_AñoY = []
    for file in parquet_files_Año:
        df = pd.read_parquet(file)
        data_frames_AñoY.append(df)

    return data_frames_AñoY


Funcion de Transformación de datos

In [42]:
def transform_data(data_frames, AñoY):
    combined_df = pd.concat(data_frames, ignore_index=True)
    combined_df['tpep_pickup_datetime'] = pd.to_datetime(combined_df['tpep_pickup_datetime'])
    combined_df['pickup_date'] = combined_df['tpep_pickup_datetime'].dt.date

    df_taxiY = combined_df.copy()
    df_taxiY['pickup_date'] = pd.to_datetime(df_taxiY['pickup_date'])

    column_order = ['VendorID', 'pickup_date'] + [col for col in df_taxiY.columns if col not in ['VendorID', 'pickup_date']]
    df_taxiY = df_taxiY[column_order]

    df_taxiY = df_taxiY[df_taxiY['pickup_date'].dt.year == AñoY]

    columns_to_remove = ['VendorID', 'Airport_fee','tpep_pickup_datetime','tpep_dropoff_datetime','RatecodeID','PULocationID','DOLocationID','passenger_count','trip_distance','fare_amount','payment_type','extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'store_and_fwd_flag', 'congestion_surcharge']
    df_taxiY = df_taxiY.drop(columns=columns_to_remove)

    # Agregar una columna que indica el día de la semana (0: Lunes, 6: Domingo)
    df_taxiY['weekday'] = df_taxiY['pickup_date'].dt.weekday

    # Mapear los números de día de la semana a nombres de días
    day_names = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    df_taxiY['weekday'] = df_taxiY['weekday'].apply(lambda x: day_names[x])

    # Agrupar por fecha y calcular el total_amount por día
    df_taxi_grouped = df_taxiY.groupby(['pickup_date', 'weekday'])['total_amount'].sum().reset_index()

    # Agregar una columna que indica la cantidad de veces que se repite una fecha
    df_taxi_grouped['row_count'] = df_taxiY.groupby('pickup_date').size().reset_index(name='row_count')['row_count']

    return df_taxi_grouped


Llamado a las funciones y guardado de los DataFrame

In [46]:
folder_path_Y = 'Datasets\\Dataset_extraidos'
AñoY = 2023
mes=5
data_frames = load_parquet_filesY(folder_path_Y, AñoY)
df_taxiY = transform_data(data_frames, AñoY)
df_taxiY.head(10)
df_taxiY.to_csv(f'taxiY_{AñoY}_{mes}.csv')

#### Combinacion taxis verdes + taxis amarillo

Función de carga de datos

In [83]:
def load_csv_files(folder_path):
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv') and f'taxiV_' in file]
    
    data_frames = []
    for file in csv_files:
        file_path = os.path.join(folder_path, file)
        df = pd.read_csv(file_path)
        data_frames.append(df)
    
    return data_frames

Función de transformación de DF

In [86]:
def combine_and_transform_data(data_frames):
    combined_df = pd.concat(data_frames, ignore_index=True)
    
    columns_to_drop = ["Unnamed: 0"]
    combined_df = combined_df.drop(columns=columns_to_drop)
    
    return combined_df

In [87]:
folder_path = 'Datasets\\Dataset_extraidos'

data_frames = load_csv_files(folder_path)
combined_data = combine_and_transform_data(data_frames)
combined_data.to_csv('taxiG.csv')

In [88]:
dfv=pd.read_csv('taxiG.csv')

In [89]:
dfv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 884 entries, 0 to 883
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    884 non-null    int64  
 1   pickup_date   884 non-null    object 
 2   weekday       884 non-null    object 
 3   total_amount  884 non-null    float64
 4   row_count     884 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 34.7+ KB


#### estaciones_ny

In [3]:
dfElectric=pd.read_csv('../Datasets/Dataset_empresa/Electric and Alternative Fuel Charging Stations.csv')

C:\Users\mwmar\AppData\Local\Temp\ipykernel_2432\3627456893.py:1: DtypeWarning: Columns (6,16,20,31,33,36,39,40,41,43,46,52,53,55,57,58,60,62) have mixed types. Specify dtype option on import or set low_memory=False.
  dfElectric=pd.read_csv('../Datasets/Dataset_empresa/Electric and Alternative Fuel Charging Stations.csv')


In [4]:
# observamos nombre de columnas, cantidad de valores no nulos y tipo de datos
dfElectric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70406 entries, 0 to 70405
Data columns (total 65 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Fuel Type Code                    70406 non-null  object 
 1   Station Name                      70406 non-null  object 
 2   Street Address                    70405 non-null  object 
 3   Intersection Directions           4493 non-null   object 
 4   City                              70406 non-null  object 
 5   State                             70406 non-null  object 
 6   ZIP                               70405 non-null  object 
 7   Plus4                             0 non-null      float64
 8   Station Phone                     65204 non-null  object 
 9   Status Code                       70406 non-null  object 
 10  Expected Date                     1218 non-null   object 
 11  Groups With Access Code           70406 non-null  object 
 12  Acce

In [5]:
dfElectric.head()

,Fuel Type Code,Station Name,Street Address,Intersection Directions,City,State,ZIP,Plus4,Station Phone,Status Code,...,EV Pricing (French),LPG Nozzle Types,Hydrogen Pressures,Hydrogen Standards,CNG Fill Type Code,CNG PSI,CNG Vehicle Class,LNG Vehicle Class,EV On-Site Renewable Source,Restricted Access
0,CNG,Spire - Montgomery Operations Center,2951 Chestnut St,NaN,Montgomery,AL,36107,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,MD,NaN,NaN,NaN
1,CNG,PS Energy - Atlanta,340 Whitehall St,"From I-7585 N, exit 91 to Central Ave, left on...",Atlanta,GA,30303,NaN,770-350-3000,E,...,NaN,NaN,NaN,NaN,Q,3600,MD,NaN,NaN,False
2,CNG,Metropolitan Atlanta Rapid Transit Authority,2424 Piedmont Rd NE,NaN,Atlanta,GA,30324,NaN,NaN,E,...,NaN,NaN,NaN,NaN,Q,3000,LD,NaN,NaN,NaN
3,CNG,United Parcel Service,270 Marvin Miller Dr,NaN,Atlanta,GA,30336,NaN,NaN,E,...,NaN,NaN,NaN,NaN,B,3600,HD,NaN,NaN,NaN
4,CNG,Arkansas Oklahoma Gas Corp,2100 S Waldron Rd,NaN,Fort Smith,AR,72903,NaN,479-783-3188,E,...,NaN,NaN,NaN,NaN,Q,3600,MD,NaN,NaN,False


In [6]:
#tranformaciones

#Filtro por ubicación
estaciones_ny = dfElectric[dfElectric['State'] == 'NY']
# Lista de nombres de columnas a eliminar
columnas_a_eliminar = ['Street Address','Intersection Directions','ZIP','Plus4','Station Phone','Status Code','Groups With Access Code',
'Access Days Time','Cards Accepted','Date Last Confirmed','Updated At','Owner Type Code','Federal Agency ID',
'Open Date','Country','Access Code','Facility Type','CNG On-Site Renewable Source','CNG Total Compression Capacity','CNG Storage Capacity','EV Pricing',
'LPG Nozzle Types','CNG Fill Type Code','CNG PSI','EV On-Site Renewable Source','Restricted Access','Expected Date','BD Blends','NG Fill Type Code','NG PSI',
'EV Other Info','EV Network Web','Hydrogen Status Link','LPG Primary', 'E85 Blender Pump', 'Intersection Directions (French)','Access Days Time (French)','BD Blends (French)',
'Hydrogen Is Retail','Federal Agency Code','LNG On-Site Renewable Source','E85 Other Ethanol Blends','EV Pricing (French)','Hydrogen Pressures','Hydrogen Standards','Federal Agency Name'
]

# Elimino las columnas especificadas
estaciones_ny = estaciones_ny.drop(columns=columnas_a_eliminar)
# Reorganizo las columnas
column_order = ['ID'] + [col for col in estaciones_ny.columns if col != 'ID']
estaciones_ny=estaciones_ny[column_order]

In [7]:
estaciones_ny.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3409 entries, 7 to 70395
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   ID                                3409 non-null   int64  
 1   Fuel Type Code                    3409 non-null   object 
 2   Station Name                      3409 non-null   object 
 3   City                              3409 non-null   object 
 4   State                             3409 non-null   object 
 5   EV Level1 EVSE Num                5 non-null      float64
 6   EV Level2 EVSE Num                3024 non-null   float64
 7   EV DC Fast Count                  241 non-null    float64
 8   EV Network                        3235 non-null   object 
 9   Geocode Status                    3409 non-null   object 
 10  Latitude                          3409 non-null   float64
 11  Longitude                         3409 non-null   float64
 12  NG Ve

In [10]:
estaciones_ny.columns.tolist()

['ID',
 'Fuel Type Code',
 'Station Name',
 'City',
 'State',
 'EV Level1 EVSE Num',
 'EV Level2 EVSE Num',
 'EV DC Fast Count',
 'EV Network',
 'Geocode Status',
 'Latitude',
 'Longitude',
 'NG Vehicle Class',
 'EV Connector Types',
 'Groups With Access Code (French)',
 'Access Detail Code',
 'CNG Dispenser Num',
 'CNG Vehicle Class',
 'LNG Vehicle Class']

In [ ]:
#guardo station_ny
estaciones_ny.to_csv('Station_NY.csv',index=False)

#### Zona de Taxis

In [ ]:
dftaxi_zone=pd.read_csv('Dataset_extraidos\\taxi_zones.csv')

In [ ]:
dftaxi_zone.info()

In [ ]:
dftaxi_zone.head()

In [ ]:
#transformaciones
columns_to_drop = ['Unnamed: 0', 'OBJECTID']
dftaxi_zone = dftaxi_zone.drop(columns=columns_to_drop)

column_order = ['LocationID'] + [col for col in dftaxi_zone.columns if col != 'LocationID']
dftaxi_zone=dftaxi_zone[column_order]

dftaxi_zone = dftaxi_zone.rename(columns={'y': 'longitude', 'x': 'latitude'})

In [ ]:
#guardo taxi zona
dftaxi_zone.to_csv('Taxi zone.csv',index=False)

#### Clima


In [ ]:
#cargo el dataset
NYCtiempo=pd.read_csv('NYCtiempo.csv')

In [ ]:
# Convertir la columna 'time' a tipo 'datetime'
NYCtiempo['time'] = pd.to_datetime(NYCtiempo['time'])
NYCtiempo['fecha'] = NYCtiempo['time'].dt.date
NYCtiempo['hora'] = NYCtiempo['time'].dt.time
NYCtiempo.drop(columns=['time'], inplace=True)

In [ ]:
NYCtiempo = NYCtiempo[['fecha', 'hora'] + [col for col in NYCtiempo.columns if col not in ['fecha', 'hora']]]

# Cambiar el nombre de la columna 'fecha' a 'time'
NYCtiempo.rename(columns={'fecha': 'time'}, inplace=True)

# Cambiar el nombre de la columna 'hora' a 'hours'
NYCtiempo.rename(columns={'hora': 'hours'}, inplace=True)

In [ ]:
# Filtrar los registros para excluir el año 2020
NYCtiempo = NYCtiempo[NYCtiempo['time'].dt.year != 2020]

# Eliminar las columnas no deseadas
columns_to_drop = ["precipitation (mm)", "rain (mm)", "is_day ()"]
NYCtiempo = NYCtiempo.drop(columns=columns_to_drop)

In [ ]:
#guardo csv
NYCtiempo.to_csv('NYCclima.csv', index=False)